<a href="https://colab.research.google.com/github/apandacoding/lumeAI/blob/main/leedai_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q langchain==0.0.150 pypdf pandas matplotlib tiktoken textract transformers openai faiss-cpu
!pip install fastapi kaleido python-multipart uvicorn cohere six>=1.13.0 beautifulsoup4>=4.11.1
!pip install -U pydantic

!pip install -U langchain
!pip install langchain-community langchain-core

import os
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import textract

from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from IPython.display import display



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
  Using cached pydantic-2.7.2-py3-none-any.whl (409 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.15
    Uninstalling pydantic-1.10.15:
      Successfully uninstalled pydantic-1.10.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.0.150 requires pydantic<2,>=1, but you have pydantic 2.7.2 which is incompatible.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have 

In [7]:
!pip install PyPDF2
import PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00


In [8]:

def combine_pdfs(pdf_list, output_path):
    """
    Combines multiple PDF files into a single PDF file.

    Parameters:
    - pdf_list: List of paths to the PDF files to be combined.
    - output_path: Path to the output combined PDF file.
    """
    # Create a PDF writer object
    pdf_writer = PyPDF2.PdfWriter()

    # Iterate through the list of PDF files
    for pdf in pdf_list:
        # Create a PDF reader object for each PDF
        pdf_reader = PyPDF2.PdfReader(pdf)

        # Add each page of the PDF to the writer object
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            pdf_writer.add_page(page)

    # Write out the combined PDF to the specified output file
    with open(output_path, 'wb') as output_pdf:
        pdf_writer.write(output_pdf)

# Example usage
if __name__ == "__main__":
    # List of PDF files to combine
    pdf_files = ['/content/AMZN-Q1-2024-Earnings-Release.pdf', '/content/Management Report Q1 2024 PDFS.pdf', '/content/new_earnings.pdf']
    # Output path for the combined PDF
    output_file = '/content/combined.pdf'

    # Combine the PDFs
    combine_pdfs(pdf_files, output_file)
    print(f"Combined PDF saved as {output_file}")

Combined PDF saved as /content/combined.pdf


In [10]:

os.environ["OPENAI_API_KEY"] = ""


loader = PyPDFLoader("/content/combined.pdf")
pages = loader.load_and_split()

# convert PDF to text

doc = textract.process("/content/combined.pdf")

# Step 2: Save to .txt and reopen (helps prevent issues)
with open('new.txt', 'w') as f:
    f.write(doc.decode('utf-8'))

with open('new.txt', 'r') as f:
    text = f.read()

# Step 3: Create function to count tokens
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

# Step 4: Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

# Get embedding model
embeddings = OpenAIEmbeddings()

# Create vector database
db = FAISS.from_documents(chunks, embeddings)


# Create QA chain to integrate similarity search with user queries (answer query from knowledge base)

chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")


# Create conversation chain that uses our vectordb as retriver, this also allows for chat history management
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.1), db.as_retriever())

chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""

    if query.lower() == 'exit':
        print("Thank you for using Leed.ai chatbot!")
        return

    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))

    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))

print("Welcome to the lume.ai chatbot! Type 'exit' to stop.")

input_box = widgets.Text(placeholder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)

Welcome to the lume.ai chatbot! Type 'exit' to stop.


Text(value='', placeholder='Please enter your question:')

HTML(value="<b>User:</b> explain Amazon's biggest moves in the last year")

HTML(value='<b><font color="blue">Chatbot:</font></b> \n1. Fast and convenient delivery: Amazon continued to d…

HTML(value='<b>User:</b> How much did operating income increase by in the first quarter of 20')

...

HTML(value='<b><font color="blue">Chatbot:</font></b>  The percent change in revenue for Linear Networks for W…

HTML(value="<b>User:</b> What was Walt Disney's loss in revenue with Direct-to-Consumer from December 2022 to …

HTML(value='<b><font color="blue">Chatbot:</font></b>  The loss in revenue for Walt Disney\'s Direct-to-Consum…

HTML(value="<b>User:</b> What was Walt Disney's Operating Loss percentage change in revenue with Direct-to-Con…

HTML(value='<b><font color="blue">Chatbot:</font></b>  The percentage change in operating loss for Walt Disney…